# Euchre
Goal: code up a playable euchre game to better understand winning probabilities of different starting hands  
Given: Right, Ace, Queen, off Ace, off ten; 3-suited, dealer spot - what's the likelihood of winning each number of tricks that round?  
Genetic Algorithm to determine next play would be cool - start with random strategy, adapt to new weights as games progress  
Streamlit app where user can select cards in hand and table position and get breakdown of likelihood of taking 5/4/3/2/1/0 tricks

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys

sys.path.insert(0, 'C:/Users/jerem/Desktop/nonsense/euchre/')

In [2]:
from utils import EuchreGame

### Create Game object, check attributes

In [3]:
euchre_game = EuchreGame()

In [4]:
euchre_game.score

{'t1': 0, 't2': 0}

In [5]:
euchre_game.dealer

'p1'

In [6]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [7]:
euchre_game.print_score()

Current score: 0-0


In [8]:
euchre_game.card_suits

['S', 'C', 'H', 'D']

In [9]:
euchre_game.card_values

['A', 'K', 'Q', 'J', 'T', '9']

In [10]:
deck_of_cards = euchre_game.shuffle_deck_of_cards()
for card in deck_of_cards:
    print(card, end=',')

9_S,9_C,9_H,9_D,T_S,T_C,T_H,T_D,J_S,J_C,J_H,J_D,Q_S,Q_C,Q_H,Q_D,K_S,K_C,K_H,K_D,A_S,A_C,A_H,A_D,

### Deal cards

In [11]:
player_hands, card_flipped_up = euchre_game.deal_hand()

None


In [12]:
player_hands

{'p1': ['Q_C', 'J_C', 'A_H', 'Q_D', 'K_S'],
 'p2': ['A_D', 'K_H', '9_H', 'T_D', 'K_C'],
 'p3': ['Q_H', 'A_C', 'J_D', '9_C', 'T_C'],
 'p4': ['9_S', 'K_D', 'Q_S', '9_D', 'J_H']}

In [13]:
card_flipped_up

'T_S'

### Choose suit for trump

In [14]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S']):
    print('order up trump')

In [15]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_S', 'J_C', 'J_S']):
    print('order up trump')

order up trump


In [16]:
euchre_game.choose_open_trump(hand=['K_S', 'A_S', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

'S'

In [17]:
euchre_game.choose_open_trump(hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

In [18]:
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                           player_hands=player_hands)

None


In [19]:
print(player_hands)
print(calling_player)
print(trump)

{'p1': ['Q_C', 'J_C', 'A_H', 'Q_D', 'K_S'], 'p2': ['A_D', 'K_H', '9_H', 'T_D', 'K_C'], 'p3': ['Q_H', 'A_C', 'J_D', '9_C', 'T_C'], 'p4': ['9_S', 'K_D', 'Q_S', '9_D', 'J_H']}
p3
C


### Play trick

In [21]:
cards_in_play, player_led = euchre_game.play_trick(player_hands=player_hands, 
                                                   trump=trump, 
                                                   next_to_play_list=euchre_game.next_to_deal, 
                                                   cards_played_this_hand = [], 
                                                   verbose=True)

Player p2 plays A_D, Player p3 plays T_C, Player p4 plays J_H, Player p1 plays K_S, 

In [22]:
cards_in_play

{'p2': 'A_D', 'p3': 'T_C', 'p4': 'J_H', 'p1': 'K_S'}

In [23]:
player_led

'p2'

#### Determine trick winner

In [42]:
euchre_game.determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_H', 'p1': 'A_S'}, 
                           trump='S', 
                           player_led=player_led, 
                                   verbose=True)

p3 wins trick


'p3'

In [47]:
euchre_game.determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_S', 'p1': 'A_S'}, 
                           trump='D', 
                           player_led=player_led, 
                                   verbose=False)

'p1'

In [46]:
trick_winner = euchre_game.determine_trick_winner(cards_in_play=cards_in_play,
                                           trump=trump,
                                           player_led=player_led)

trick_winner

'p3'

#### Swap card in dealer hand

In [171]:
new_dealer_hand = euchre_game.swap_dealer_card(card_flipped_up='A_S', dealer_hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
                                               verbose=True)
new_dealer_hand

Dealer discards 9_H and picks up A_S


['J_H', 'A_D', 'K_S', 'J_D', 'A_S']

In [172]:
new_dealer_hand = euchre_game.swap_dealer_card(card_flipped_up='9_S', dealer_hand=['J_H', 'T_D', 'T_H', 'K_S', 'J_D'], 
                                               verbose=True)
new_dealer_hand

Dealer discards T_D and picks up 9_S


['J_H', 'T_H', 'K_S', 'J_D', '9_S']

### Play card

In [30]:
euchre_game.play_card(hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
                      trump='S',
                      cards_in_play=['K_H', 'T_H'], 
                      cards_played_this_hand=[], 
                      suit_led='H')

'J_D'

In [32]:
euchre_game.play_card(hand=['9_D', 'T_C', '9_C', 'J_H', 'A_C'],
                      trump='S',
                      cards_in_play=[],
                      cards_played_this_hand=[], 
                      suit_led=None)

'A_C'

In [34]:
euchre_game.play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=[],
                      cards_played_this_hand=[], 
          suit_led=None)

'J_S'

In [35]:
euchre_game.play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=['K_C'],
                                cards_played_this_hand=[], 
          suit_led=None)

'A_C'

In [93]:
euchre_game.play_random_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                             suit_led=None)

'J_S'

In [94]:
euchre_game.play_random_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                             suit_led='D')

'9_D'

In [97]:
euchre_game.play_random_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                             suit_led='C')

'9_C'

In [ ]:
def play_card(self,
              hand,
              trump,
              cards_in_play,
              player_led,
              cards_played_this_hand,
              suit_led=None):
    """
    Function to return card to play in hand
    """
    card_values = {
        '9': 1,
        'T': 2,
        'J': 3,
        'Q': 4,
        'K': 5,
        'A': 6
    }
    # play last card
    if len(hand) == 1:
        return hand[0]

    # lead card
    if len(cards_in_play) < 1:
        for idx, card in enumerate(hand):
            # 1 - play right bauer
            if card[-1] == trump and card[0] == 'J':
                return hand[idx]
            # TODO: 1A - if right bauer played, play left bauer
            right_bauer = 'J' + '_' + trump
            if right_bauer in cards_played_this_hand:
                pass
            # 2 - play off ace
            elif card[-1] != trump:
                if card[0] == 'A':
                    return hand[idx]
            # 3 - TODO: else play highest non-trump card
        return hand[0]

    # follow suit
    if suit_led is not None:
        current_winning_player = euchre_game.determine_trick_winner(cards_in_play=cards_in_play,
                                                                    trump=trump,
                                                                    player_led=player_led)
        # play lowest card in the suit played
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] == suit_led:
                card_points = card_values[card[0]]
                if card_points < card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        return hand[idx_to_return]

    # play other highest non-trump card
    else:
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] != trump:
                card_points = card_values[card[0]]
                if card_points > card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        if idx_to_return > -1:
            return hand[idx_to_return]

        else:  # only has trump left, play lowest trump
            trump_card_points = 9
            for idx, card in enumerate(hand):
                card_points = card_values[card[0]]
                if card_points < trump_card_points:
                    trump_card_points = card_points
                    idx_to_return = idx
            return hand[idx_to_return]


In [ ]:
# play card pseudocode:
    # last card
    # follow suit
    # play whatever card

In [226]:
card_values = {
    '9': 1,
    'T': 2,
    'J': 3,
    'Q': 4,
    'K': 5,
    'A': 6
}

In [247]:
card_values[card[0]]

6

In [246]:
card

'A_D'

In [233]:
def get_lowest_card(hand, 
                    suit=None):
    """
    Return lowest card in given suit or across all suits
    """
    card_to_play_points = 9
    idx_to_return = -1
    if suit is not None:
        for idx, card in enumerate(hand):
            if card[-1] == suit:  # if card is in given suit
                card_points = card_values[card[0]]  
                if card_points < card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        if idx_to_return > -1:
            return hand[idx_to_return]
    else:
        for idx, card in enumerate(hand):
            card_points = card_values[card[0]]  
            if card_points < card_to_play_points:
                card_to_play_points = card_points
                idx_to_return = idx
        return hand[idx_to_return]

In [234]:
get_lowest_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                suit='C')

'9_C'

In [235]:
get_lowest_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                suit='H')

In [237]:
get_lowest_card(hand=['A_D', 'T_C', '9_C', 'J_S', 'A_C'], 
                suit='D')

'A_D'

In [240]:
get_lowest_card(hand=['A_D', 'T_C', '9_C', 'J_S', 'A_C'])

'9_C'

In [243]:
get_lowest_card(hand=['A_D', 'T_C', 'T_H', 'J_S', 'A_C'])

'T_C'

### Play full game

In [263]:
%%time

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=False)

Wall time: 13 ms


In [264]:
euchre_game.score

{'t1': 10, 't2': 4}

#### Collect all hands results

In [209]:
euchre_game = EuchreGame()
hand_results = euchre_game.play_full_game(return_all_hands_results=True, verbose=False)
hand_results[0:3]

[{'player_hands': {'p1': ['J_C', 'K_S', 'A_S', '9_C', 'T_C'],
   'p2': ['9_H', '9_S', 'A_H', 'A_C', 'Q_H'],
   'p3': ['Q_C', 'K_C', 'J_S', 'A_D', 'Q_D'],
   'p4': ['K_D', 'K_H', 'J_D', '9_D', 'T_S']},
  'calling_player': 'p2',
  'trump': 'H',
  'dealer': 'p1',
  'trick_winners': {'p2': 2, 'p3': 0, 'p4': 2, 'p1': 1}},
 {'player_hands': {'p1': ['A_H', 'J_D', 'Q_C', 'T_D', 'J_H'],
   'p2': ['K_H', '9_H', 'T_C', 'T_S', 'Q_D'],
   'p3': ['J_S', 'K_D', 'A_D', 'A_C', '9_S'],
   'p4': ['T_H', 'A_S', 'K_C', 'Q_S', 'K_S']},
  'calling_player': 'p4',
  'trump': 'S',
  'dealer': 'p2',
  'trick_winners': {'p3': 3, 'p4': 2, 'p1': 0, 'p2': 0}},
 {'player_hands': {'p1': ['K_D', 'T_S', 'T_C', 'J_S', 'A_S'],
   'p2': ['9_C', 'Q_D', 'K_H', 'A_H', 'Q_H'],
   'p3': ['J_C', '9_H', 'A_C', '9_D', 'T_D'],
   'p4': ['J_D', 'Q_S', 'T_H', 'K_S', 'A_D']},
  'calling_player': 'p1',
  'trump': 'S',
  'dealer': 'p3',
  'trick_winners': {'p4': 1, 'p1': 3, 'p2': 0, 'p3': 1}}]

In [212]:
len(hand_results)

19

In [211]:
hand_results[2]

{'player_hands': {'p1': ['K_D', 'T_S', 'T_C', 'J_S', 'A_S'],
  'p2': ['9_C', 'Q_D', 'K_H', 'A_H', 'Q_H'],
  'p3': ['J_C', '9_H', 'A_C', '9_D', 'T_D'],
  'p4': ['J_D', 'Q_S', 'T_H', 'K_S', 'A_D']},
 'calling_player': 'p1',
 'trump': 'S',
 'dealer': 'p3',
 'trick_winners': {'p4': 1, 'p1': 3, 'p2': 0, 'p3': 1}}

#### Profile performance

In [175]:
%%prun

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=False)

### Loop through games played

In [176]:
%%time

N_GAMES_TO_PLAY = 1000

games_played = 0
t1_wins = 0
for i in range(N_GAMES_TO_PLAY):
    euchre_game = EuchreGame()
    # euchre_game.team_strategies['t2'] = 'random'  # set team 2 strategy to random
    euchre_game.play_full_game(verbose=False)
    games_played += 1
    if euchre_game.score['t1'] > euchre_game.score['t2']:
        t1_wins += 1

Wall time: 6.52 s


In [177]:
print(games_played)
print(t1_wins)

1000
486


### Loop through games played, collect hand_results

In [213]:
%%time

N_GAMES_TO_PLAY = 1000

all_hand_results = []
games_played = 0
t1_wins = 0
for i in range(N_GAMES_TO_PLAY):
    euchre_game = EuchreGame()
    # euchre_game.team_strategies['t2'] = 'random'  # set team 2 strategy to random
    hand_results = euchre_game.play_full_game(return_all_hands_results=True, verbose=False)
    games_played += 1
    if euchre_game.score['t1'] > euchre_game.score['t2']:
        t1_wins += 1
    all_hand_results.append(hand_results)

Wall time: 9.05 s


In [219]:
len(all_hand_results)

1000

In [220]:
all_hand_results[0][0]

{'player_hands': {'p1': ['Q_D', 'T_H', 'Q_S', 'K_C', 'K_D'],
  'p2': ['A_H', 'A_D', '9_S', 'A_S', 'K_H'],
  'p3': ['9_H', 'T_D', 'J_S', 'J_C', '9_D'],
  'p4': ['T_S', 'J_H', '9_C', 'T_C', 'Q_C']},
 'calling_player': 'p4',
 'trump': 'C',
 'dealer': 'p1',
 'trick_winners': {'p2': 0, 'p3': 2, 'p4': 3, 'p1': 0}}

### Next steps for development

In [ ]:
# build process to evaluate points scored for each hand type for each game
# build out play card strategy
    # add logic to check what cards have been played in hand so far
        # check what trump has been played, etc., if highest remaining trump, play that
# build out strategy on when to call trump
    # build out code framework to properly evaluate this?
# Handle loners
# build out advanced strategy for dealer swapping out card_flipped_up to reduce number of suits in hand
# Code cleanup
    # Add README
    # Add tests to all functions
    # Fix static methods